In [1]:
%pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.4 MB/s eta 0:00:00


In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#%pip install catboost
from catboost import Pool
import catboost as cb
from catboost import CatBoostRegressor

In [28]:
def check_nan(df):# сколько где nan в процентах
    result = df.isnull().sum() * 100/ len(df)
    result = result[result > 0].sort_values()
    return result

In [29]:
df = pd.read_csv('train.csv') # читаем данные
df
check_nan(df)

Electrical       0.068493
MasVnrType       0.547945
MasVnrArea       0.547945
BsmtQual         2.534247
BsmtCond         2.534247
BsmtFinType1     2.534247
BsmtExposure     2.602740
BsmtFinType2     2.602740
GarageCond       5.547945
GarageQual       5.547945
GarageFinish     5.547945
GarageType       5.547945
GarageYrBlt      5.547945
LotFrontage     17.739726
FireplaceQu     47.260274
Fence           80.753425
Alley           93.767123
MiscFeature     96.301370
PoolQC          99.520548
dtype: float64

In [30]:
default_dict = {
    'MSSubClass' : 60,'MSZoning' : 'RL','LotArea' : 5000,'Street' : 'Pave'
    ,'LotShape' : 'Reg','LandContour' : 'Lvl','Utilities' : 'AllPub','LotConfig' : 'Inside'
    ,'LandSlope' : 'Gtl','Neighborhood' : 'CollgCr','Condition1' : 'Norm','BldgType' : '1Fam'
    ,'HouseStyle' : '2Story','OverallQual' : 7,'OverallCond' : 8,'YearBuilt' : 1965,'YearRemodAdd' : 2003
    ,'RoofStyle' : 'Gable','RoofMatl' : 'CompShg','Exterior1st' : 'VinylSd','Exterior2nd' : 'VinylSd',
    'ExterQual' : 'Gd','ExterCond' : 'TA','Foundation' : 'PConc','BsmtFinSF1' : 578,'BsmtFinSF2' : 668,
    'BsmtUnfSF' : 434 ,'HeatingQC' : 'Ex','CentralAir' : 'Y','1stFlrSF' : 1022,'2ndFlrSF' : 756,
    'LowQualFinSF'  : 513,'GrLivArea' : 1262,'BsmtFullBath' : 1,'BsmtHalfBath' : 1,'FullBath' : 2,
    'HalfBath' : 1,'BedroomAbvGr' : 3,'KitchenAbvGr' : 2,'KitchenQual' : 'Gd','TotRmsAbvGrd' : 9,
    'Functional' : 'Typ','Fireplaces' : 1,'GarageArea' : 548,'PavedDrive' : 'Y','WoodDeckSF' : 298,
    'OpenPorchSF' : 213,'EnclosedPorch' : 272,'3SsnPorch' : 320,'ScreenPorch' : 410,'PoolArea' : 648,
    'MiscVal' : 8300,'YrSold' : 2005
}

In [10]:
new_lst = []

In [11]:
for item in df.columns:
  if item not in default_dict.keys():
    new_lst.append(item)

In [16]:
y = df[df.columns[-1]]

In [18]:
new_df = df.drop(new_lst,axis = 1)

In [19]:
cat_features = []
for item in new_df.columns:
    if type(new_df[item][0]) == str:
        new_df[item].fillna('other',inplace = True)
        cat_features.append(item)

In [20]:
train_dataset = cb.Pool(new_df,y,
                        cat_features=cat_features)

In [23]:
model = CatBoostRegressor(iterations=10000,cat_features=cat_features,eval_metric = 'RMSE')

In [ ]:
model.fit(train_dataset, plot = True,verbose = True)

In [25]:
import pickle

In [26]:
with open("kekboost.pickle", "wb") as file:
    pickle.dump(model, file)

In [33]:
model.predict(list(default_dict.values()))

218855.6135092111

In [39]:
test_df = pd.read_csv('test.csv')

In [41]:
new_test_df = test_df.drop(new_lst[:-1],axis = 1)

In [40]:
predict = pd.DataFrame(test_df['Id'])

In [42]:
new_test_df

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,ExterQual,ExterCond,Foundation,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,HeatingQC,CentralAir,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageArea,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,YrSold
0,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,TA,TA,CBlock,468.0,144.0,270.0,TA,Y,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,730.0,Y,140,0,0,0,120,0,0,2010
1,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,TA,TA,CBlock,923.0,0.0,406.0,TA,Y,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,312.0,Y,393,36,0,0,0,0,12500,2010
2,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,TA,TA,PConc,791.0,0.0,137.0,Gd,Y,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,482.0,Y,212,34,0,0,0,0,0,2010
3,60,RL,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,TA,TA,PConc,602.0,0.0,324.0,Ex,Y,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,470.0,Y,360,36,0,0,0,0,0,2010
4,120,RL,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,Gd,TA,PConc,263.0,0.0,1017.0,Ex,Y,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,506.0,Y,0,82,0,0,144,0,0,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,1936,Pave,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,TA,TA,CBlock,0.0,0.0,546.0,Gd,Y,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,0.0,Y,0,0,0,0,0,0,0,2006
1455,160,RM,1894,Pave,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,TA,TA,CBlock,252.0,0.0,294.0,TA,Y,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,286.0,Y,0,24,0,0,0,0,0,2006
1456,20,RL,20000,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,TA,TA,CBlock,1224.0,0.0,0.0,Ex,Y,1224,0,0,1224,1.0,0.0,1,0,4,1,TA,7,Typ,1,576.0,Y,474,0,0,0,0,0,0,2006
1457,85,RL,10441,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,1Fam,SFoyer,5,5,1992,1992,Gable,CompShg,HdBoard,Wd Shng,TA,TA,PConc,337.0,0.0,575.0,TA,Y,970,0,0,970,0.0,1.0,1,0,3,1,TA,6,Typ,0,0.0,Y,80,32,0,0,0,0,700,2006


In [38]:
new_df

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,ExterQual,ExterCond,Foundation,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,HeatingQC,CentralAir,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageArea,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,YrSold
0,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,Gd,TA,PConc,706,0,150,Ex,Y,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,548,Y,0,61,0,0,0,0,0,2008
1,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,TA,TA,CBlock,978,0,284,Ex,Y,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,460,Y,298,0,0,0,0,0,0,2007
2,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,Gd,TA,PConc,486,0,434,Ex,Y,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,608,Y,0,42,0,0,0,0,0,2008
3,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,TA,TA,BrkTil,216,0,540,Gd,Y,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,642,Y,0,35,272,0,0,0,0,2006
4,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,Gd,TA,PConc,655,0,490,Ex,Y,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,836,Y,192,84,0,0,0,0,0,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,TA,TA,PConc,0,0,953,Ex,Y,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,460,Y,0,40,0,0,0,0,0,2007
1456,20,RL,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,TA,TA,CBlock,790,163,589,TA,Y,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,500,Y,349,0,0,0,0,0,0,2010
1457,70,RL,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,Ex,Gd,Stone,275,0,877,Ex,Y,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,252,Y,0,60,0,0,0,0,2500,2010
1458,20,RL,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,TA,TA,CBlock,49,1029,0,Gd,Y,1078,0,0,1078,1,0,1,0,2,1,Gd,5,Typ,0,240,Y,366,0,112,0,0,0,0,2010


In [44]:
for item in new_test_df.columns:
    if type(new_test_df[item][0]) == str:
        new_test_df[item].fillna('other',inplace = True)

In [46]:
pred = model.predict(new_test_df)

In [47]:
predict['SalePrice'] = pred

In [48]:
predict.to_csv('submit.csv',index = False)